<a href="https://colab.research.google.com/github/pedrohortencio/text-generator/blob/main/Poetry%20Generator/Generate_a_Random_Poetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construção do Modelo

As duas células dessa sessão precisam ser executadas. Não precisa expandir e executar elas individualmente, basta apertar o _play button_ logo abaixo:

In [1]:
import urllib
import urllib.request
import zipfile
import os
import pickle
import numpy as np

from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences


# Pre-trained Model URL
url_model = "https://github.com/pedrohortencio/text-generator/blob/main/Poetry%20Generator/model-data/model-poetry-generator-ptbrV2.02.25041118.zip?raw=true"
# Tokenizer URL
url_tokenizer = 'https://github.com/pedrohortencio/text-generator/blob/main/Poetry%20Generator/model-data/tokenizer.pickle?raw=true'

# Download the tokenizer and model
filename, headers = urllib.request.urlretrieve(url_model, filename="/content/model.zip")
filename, headers = urllib.request.urlretrieve(url_tokenizer, filename="/content/tokenizer.pickle")

# Unzip the model
with zipfile.ZipFile('/content/model.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/model')

# Loads the tokenizer
with open('/content/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [2]:
def predictions(seed_text, next_words=50):

    max_sequence_len = 12

    # Loads the model
    model = keras.models.load_model('/content/model/content/model')

    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        #predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word

    seed_text = seed_text.replace(" .", ".").replace(" ,", ",").replace(" !", "!").replace(" ?", "?").replace(" :", ":")

    return seed_text

# Criação de (Quase) Poesias

In [ ]:
print("Versão do Modelo: 2.01.0604.1841")
print("--"*20)
print("O gerador de poesias funciona com dois dados: seed e quantidade de palavras a serem geradas")
print("A seed é apenas uma frase curta que será o início da poesia. Sinta-se à vontade para digitar qualquer coisa.")
print("\tNota: o modelo ainda está em fase de refinamento")
print("--"*20, "\n")

while True:
    print("Seed")
    seed = input("Digite uma frase curta: ")
    try:
        print("Número de palavras a serem geradas")
        num_words = int(input())
    except:
        num_words = 50

    print("Gerando poesia\n", "..."*10)

    poesia = predictions(seed, num_words)

    print("Poesia gerada:\n\n", poesia, "\n")
    print("..."*10)

    print('Prosseguir? (Digite "pare" para encerrar o loop)')
    check = input()

    if check.upper() == "PARE":
        break